In [86]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
from config import path_to_repository
from analysis import BallBearingData # class for extracting all data from application and reapplication for ADC, IMU, and Camera Triggers
from analysis import DLC3DBendAngles # class for taking DLC 3d point data and converting to angles 
from analysis import bender_class # class for normalizing adc data, analyzing autobender tests
from analysis_adc_cam import ADC_CAM
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [88]:
# Upload ball bearing and beaker Trials from particpant
# Upload only the parent *date* folder (e.g., "9_19_25").
# If something fails, verify your folder structure matches the "9_19_25" example.

root_dir="CSV Data/11_2_25"
folder_suffix="B1_slow"

#10_28_25 (Tom)
#10_29_25 (bending data might be okay...check)
#11_2_25 (good)
#11_4_25 (good)

##################### raw adc or imu data for any test ###############################

folder_suffix_first  = "B1_slow"       # e.g.  9_19_25_B1_slow
folder_suffix_second = "B2_slow"       # e.g.  9_19_25_B2_slow

cam = ADC_CAM(
    root_dir=root_dir,
    path_to_repo=path_to_repository,
    folder_suffix_first="B1_slow",
    folder_suffix_second="B2_slow",
)

first_trials  = cam.load_first()
second_trials = cam.load_second()

n_first  = len(first_trials)
n_second = len(second_trials)

print("Number of first trials:", n_first)
print("Number of second trials:", n_second)


Number of first trials: 5
Number of second trials: 3


In [89]:
adc_trials_first  = cam.extract_adc_dfs_by_trial(first_trials)
adc_trials_second = cam.extract_adc_dfs_by_trial(second_trials)

print("Number of first ADC trials:", len(adc_trials_first))
print("Number of second ADC trials:", len(adc_trials_second))


Number of first ADC trials: 5
Number of second ADC trials: 3


In [90]:
adc_cal_df = cam.extract_calib_means_by_set(
    adc_column="adc_ch3",             # or None to auto-pick
    exclude_name_contains=("C_Block",),
    exclude_sets=(3, 4),
    make_plot=False,
    overlay_mean=False,
    point_alpha=0.15,                 # maybe lower alpha for all points
    point_size=5,                     # smaller points for dense clouds
    jitter=0.25,
    snap_tol_deg=4.0,
    plot_all_data=True,               # <-- THIS plots all ADC samples
)



In [112]:
out_cam = cam.calibrate_trials_with_camera(
    adc_trials_first=adc_trials_first,
    adc_trials_second=adc_trials_second,
    adc_column="adc_ch3",
    poly_order=2,
    calib_kwargs=dict(
        adc_column="adc_ch3",
        exclude_name_contains=("C_Block",),
        exclude_sets=(3, 4),
        make_plot=False,
        overlay_mean=False,
        point_alpha=0.25,
        point_size=10,
        jitter=0.25,
        snap_tol_deg=4.0,
    ),
    clamp_theta=True,   # <--- enable clamping
    deg_min=0.0,
    deg_max=90.0,
)

adc_first_theta  = out_cam["adc_trials_first_theta"]
adc_second_theta = out_cam["adc_trials_second_theta"]



,timestamp,adc_ch0,adc_ch1,adc_ch2,adc_ch3,trial_index,source_path,theta_cam_cal
0,142738116336,0,0,0,9149598,0,C:\Users\toppe\OneDrive - CSU Maritime Academy...,0.0
1,142738116336,0,0,0,9150281,0,C:\Users\toppe\OneDrive - CSU Maritime Academy...,0.0
2,142738116336,0,0,0,9143796,0,C:\Users\toppe\OneDrive - CSU Maritime Academy...,0.0
3,142738116336,0,0,0,9148404,0,C:\Users\toppe\OneDrive - CSU Maritime Academy...,0.0
4,142738116336,0,0,0,9150452,0,C:\Users\toppe\OneDrive - CSU Maritime Academy...,0.0
...,...,...,...,...,...,...,...,...
6945,142748116024,0,0,0,9128095,0,C:\Users\toppe\OneDrive - CSU Maritime Academy...,0.0
6946,142748116024,0,0,0,9133726,0,C:\Users\toppe\OneDrive - CSU Maritime Academy...,0.0
6947,142748116024,0,0,0,9135092,0,C:\Users\toppe\OneDrive - CSU Maritime Academy...,0.0
6948,142748116024,0,0,0,9127071,0,C:\Users\toppe\OneDrive - CSU Maritime Academy...,0.0


# Get per-trial camera timestamp tables from flir.mat (prefix 'ts*') and DLC 3D point data (two separate dataframes)

This is where you should probably also load the DLC data and combine with cam_trials_first and cam_trials_second.    See my BallBearingClass for examples of how to do that.  